In [1]:
!pip install langchain requests langchain-community langchain-experimental pydantic-ai-slim[duckduckgo] langchain-google-genai

In [2]:
import os
from getpass import getpass
from langchain.tools import tool
import requests
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate


In [4]:
# Set up API keys
weather_api_key = getpass("Enter your WeatherAPI.com key: ")
os.environ["WEATHER_API_KEY"] = weather_api_key
os.environ["GOOGLE_API_KEY"] = getpass("Enter your Gemini API key: ")

Enter your WeatherAPI.com key: ··········
Enter your Gemini API key: ··········


In [5]:
# Initialize Gemini LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.7)

In [6]:
# Custom Weather Tool
@tool
def get_weather_forecast(location: str) -> str:
    """Fetches the current weather forecast for a given location."""
    api_key = os.environ.get("WEATHER_API_KEY")
    url = f"http://api.weatherapi.com/v1/current.json?key={api_key}&q={location}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        weather = data["current"]
        return f"Current weather in {location}: {weather['condition']['text']}, Temperature: {weather['temp_c']}°C, Humidity: {weather['humidity']}%"
    except requests.exceptions.RequestException as e:
        return f"Error fetching weather for {location}: {str(e)}"

In [7]:
# DuckDuckGo Search Tool
search_tool = DuckDuckGoSearchRun()

# Combine tools
tools = [get_weather_forecast, search_tool]

In [8]:
# Define agent prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a travel assistant AI. Use the provided tools to fetch weather forecasts and top tourist attractions for the user's destination. Summarize the results clearly."),
    ("human", "Provide the weather forecast and top tourist attractions for {destination}."),
    ("placeholder", "{agent_scratchpad}")
])

In [9]:
# Create and execute agent
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# User input
destination = input("Enter your destination city: ")
result = agent_executor.invoke({"destination": destination})
print(result["output"])

Enter your destination city: Coimbatore


> Entering new AgentExecutor chain...

Invoking: `get_weather_forecast` with `{'location': 'Coimbatore'}`


Current weather in Coimbatore: Light rain, Temperature: 27.3°C, Humidity: 79%
Invoking: `duckduckgo_search` with `{'query': 'top tourist attractions in Coimbatore'}`


Some of the attractions that a traveller is most likely to come across include the following; Temples, Waterfalls, and Mountains among others. Below are a few useful tips on some of the tourist attractions that are worth visiting in Coimbatore. 1.Adiyogi Shiva Statue. Location: Isha Yoga Center - Velliangiri Foothills 20. Race Course Walking Trail- Tourist Places in Coimbatore. A perfect place for fitness enthusiasts, this well-maintained walking trail is lined with trees and benches. Address: Race Course Road, Coimbatore; Timings: Open 24 hours; 21. Insect Museum, TNAU- Tourist Places in Coimbatore. A unique museum showcasing a vast collection of insect ... For more cultur